## Tabular training

In [36]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib pandas

To illustrate the tabular application, we will use the example of the Adult dataset where we have to predict if a person is earning more or less than $50k per year using some general data.

In [37]:
from fastai.tabular.all import *
from pathlib import Path
import pandas as pd

## Variables

In [38]:
modelName = 'stockScreenerV2.pkl'
trainingDataName = 'stockData.csv'
trainingFolder = Path.cwd().parent / 'TrainingData'
modelFolder = Path.cwd().parent.parent / 'TrainedModels'

# Training parameters
yNames = ['Future Year Change']
catNames = []
contNames = [
    'Open',
    'High', 
    'Low', 
    'Close', 
    'Volume', 
    'Dividends', 
    'Stock Splits', 
    'EV/EBIT', 
    'Market Cap', 
    'ROIC'
]

We can download a sample of this dataset with the usual untar_data command:

In [39]:
dataPath = Path()
dataPath.ls()

(#3) [Path('app.ipynb'),Path('stockFetcher.ipynb'),Path('tabular.ipynb')]

Then we can have a look at how the data is structured:

In [40]:
df = pd.read_csv(trainingFolder/trainingDataName)
df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,Future Year Change,EV/EBIT,Market Cap,ROIC
0,2019-01-17 00:00:00-05:00,AAPL,36.820321,37.646511,36.595865,37.216702,119284800,0.0,0.0,1.075703,10.460258,5.596610e+11,0.075524
1,2019-01-18 00:00:00-05:00,AAPL,37.608309,37.699047,37.245358,37.445938,135004000,0.0,0.0,1.049015,10.519029,5.631083e+11,0.075102
2,2019-01-22 00:00:00-05:00,AAPL,37.348034,37.424443,36.443045,36.605419,121576000,0.0,0.0,1.103545,10.303539,5.504686e+11,0.076673
3,2019-01-23 00:00:00-05:00,AAPL,36.808383,37.044779,36.223365,36.753464,92522400,0.0,0.0,1.105162,10.341494,5.526949e+11,0.076391
4,2019-01-24 00:00:00-05:00,AAPL,36.798832,36.887180,36.232917,36.462147,101766000,0.0,0.0,1.115866,10.266807,5.483141e+11,0.076947


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in TabularDataLoaders factory methods:

In [41]:
dls = TabularDataLoaders.from_csv(trainingFolder/trainingDataName, path=dataPath, 
    y_names=yNames,
    cat_names=catNames,
    cont_names=contNames,
    procs = [Categorify, FillMissing, Normalize])

The last part is the list of pre-processors we apply to our data:

* Categorify is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
* FillMissing will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
* Normalize will normalize the continuous variables (subtract the mean and divide by the std)

To further expose what’s going on below the surface, let’s rewrite this utilizing fastai’s TabularPandas class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

In [42]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [43]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
    y_names=yNames,
    cat_names = catNames,
    cont_names = contNames,
    splits=splits)

Once we build our TabularPandas object, our data is completely preprocessed as seen below:

In [44]:
to.xs.iloc[:2]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC
45755,0.129526,0.129106,0.089556,0.085959,-0.278942,-0.048962,-0.009378,3.034491,-0.438477,-0.014056
35663,-0.459653,-0.462141,-0.459016,-0.460777,-0.244515,-0.048962,-0.009378,-0.516069,-0.387066,0.000401


Now we can build our DataLoaders again:

In [45]:
dls = to.dataloaders(bs=64)

The show_batch method works like for every other application:

In [46]:
dls.show_batch()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change
0,324.585298,327.293699,323.745090,326.305237,1.774150e+07,-8.591427e-12,-1.386151e-11,64.110817,2.426040e+12,0.012322,0.216403
1,33.964611,34.139416,32.656688,32.925473,2.079600e+07,-8.591427e-12,-1.386151e-11,17.651377,4.236489e+10,0.044756,0.774294
2,112.852897,116.410030,112.852894,116.180546,1.567600e+06,-8.591427e-12,-1.386151e-11,12.493068,3.833133e+10,0.063235,-0.223166
3,50.199845,50.429186,49.908755,50.340978,2.005110e+07,-8.591427e-12,-1.386151e-11,29.896662,2.171206e+11,0.026424,-0.097429
4,87.104240,87.826625,86.838214,87.045961,3.183800e+07,-8.591427e-12,-1.386151e-11,8.722608,5.086095e+11,0.090569,0.668544
5,79.600870,80.445561,79.392071,79.790685,6.570100e+06,-8.591427e-12,-1.386151e-11,20.773617,9.046668e+10,0.038029,0.190451
6,146.999998,147.294999,140.447998,142.585009,1.529100e+07,-8.591427e-12,-1.386151e-11,137.341599,1.729314e+11,0.005752,-0.798366
7,212.362381,216.436737,212.362381,214.100525,2.143301e+06,-8.591427e-12,-1.386151e-11,28.229540,1.392182e+11,0.027985,-0.120419
8,83.339993,86.919998,82.249996,85.480003,2.196401e+06,-8.591427e-12,-1.386151e-11,33.763370,2.105467e+10,0.023398,0.344876
9,289.809996,291.140013,286.709991,287.320003,1.976600e+06,-8.591427e-12,-1.386151e-11,43.443210,7.399323e+10,0.018185,0.220869


We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.

Note: Sometimes with tabular data, your y’s may be encoded (such as 0 and 1). In such a case you should explicitly pass y_block = CategoryBlock in your constructor so fastai won’t presume you are doing regression.

In [47]:
learn = tabular_learner(dls, metrics=[rmse, mae])

And we can train that model with the fit_one_cycle method (the fine_tune method won’t be useful here since we don’t have a pretrained model).

In [48]:
learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,_rmse,mae,time
0,0.397668,1.136165,1.065910,0.421423,00:08
1,0.377991,27.006987,5.196825,0.426404,00:08
2,0.357174,6.038046,2.457243,0.409335,00:08
3,0.366437,4468.829590,66.849304,1.102603,00:08
4,0.371063,6.836284,2.614628,0.412395,00:08
5,0.344427,0.659740,0.812244,0.383329,00:08
6,0.375583,11502.334961,107.248901,1.558986,00:08
7,0.389309,1023.682556,31.995047,0.758212,00:07
8,0.328159,20359660.000000,4512.167969,48.848125,00:07
9,0.351078,28513.056641,168.858154,2.407831,00:07


We can then have a look at some predictions:

In [49]:
learn.show_results()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change,Future Year Change_pred
0,-0.173333,-0.179557,-0.175874,-0.177871,-0.316111,-0.048962,-0.009378,0.858704,-0.377746,-0.010736,0.116062,0.222963
1,-0.572933,-0.574070,-0.572926,-0.574467,1.713824,-0.048962,-0.009378,-1.234011,-0.409430,0.121329,0.596349,1.892530
2,-0.160291,-0.108820,-0.158208,-0.103567,4.388946,-0.048962,36.623192,0.132159,0.735038,-0.007443,0.476401,170778.625000
3,-0.348766,-0.350194,-0.349237,-0.352763,2.225235,-0.048962,-0.009378,-0.359856,2.626174,-0.002438,0.492073,0.495378
4,-0.366967,-0.355967,-0.368983,-0.352451,-0.160805,-0.048962,-0.009378,0.119305,-0.496175,-0.007355,1.529894,0.474776
5,-0.347184,-0.350786,-0.343159,-0.346603,0.229359,-0.048962,-0.009378,-0.883764,1.720466,0.014763,0.788355,0.386501
6,-0.049323,-0.051592,-0.046948,-0.048565,-0.316965,-0.048962,-0.009378,-0.088869,-0.453766,-0.005681,0.483234,0.110937
7,-0.220113,-0.219580,-0.214768,-0.214024,-0.106206,-0.048962,-0.009378,-0.795287,1.201265,0.009535,0.230481,0.232463
8,0.005394,0.019274,-0.002736,-0.005302,-0.192319,-0.048962,-0.009378,2.522104,-0.365761,-0.013611,-0.493960,-0.176445


## Export the model

In [50]:
learn.export(modelFolder / f'{modelName}')


To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. That dataframe does not need to have the dependent variable in its column.

In [ ]:
from ipynb.fs.full.stockFetcher import getTickerData

PLTR: Period '10y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']


Error processing PLTR: index -1 is out of bounds for axis 0 with size 0
Static Data for AAPL:
  Total Debt: 119058997248
  Cash: 65171001344
  Shares Outstanding: 15037899776
Approximated EBIT for AAPL: 58655249203.2
Static Data for MSFT:
  Total Debt: 96838000640
  Cash: 78429003776
  Shares Outstanding: 7434880000
Approximated EBIT for MSFT: 38128499097.6
Static Data for AMZN:
  Total Debt: 158534991872
  Cash: 88050999296
  Shares Outstanding: 10515000320
Approximated EBIT for AMZN: 93019196620.8
Static Data for GOOGL:
  Total Debt: 29289000960
  Cash: 93229998080
  Shares Outstanding: 5842999808
Approximated EBIT for GOOGL: 50978852044.799995
Static Data for GOOG:
  Total Debt: 29289000960
  Cash: 93229998080
  Shares Outstanding: 5534000128
Approximated EBIT for GOOG: 50978852044.799995
Static Data for META:
  Total Debt: 49046999040
  Cash: 70899998720
  Shares Outstanding: 2180000000
Approximated EBIT for META: 23434049126.399998
Static Data for TSLA:
  Total Debt: 12782999552
 

In [54]:
predictionTarget = 'TSLA'

test_df = getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
	test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)

prediction = learn.get_preds(dl=dl)
print(f"Predicted {yNames[0]} for {predictionTarget}:")
print(f"{prediction[0][0][0].item() * 100:.2f}%")

Static Data for TSLA:
  Total Debt: 12782999552
  Cash: 33648001024
  Shares Outstanding: 3210060032
Approximated EBIT for TSLA: 14572500172.8


Predicted Future Year Change for TSLA:
-37.76%


Note:
Since machine learning models can’t magically understand categories it was never trained on, the data should reflect this. If there are different missing values in your test data you should address this before training